In [86]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [87]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

In [88]:
data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush.csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    # add queried data (without cost) to initial data
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)

    # data = data.append(queried_data, ignore_index=True)
    return data

data = add_data(data, df_queried)
data = data.reset_index().astype('Float32')

print(data)

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

     index  Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
0      0.0         700.0         7.33    500.899994   78.375298         1.91   
1      1.0         700.0        25.85    577.599976  118.067299         0.94   
2      2.0         700.0    46.669998    615.799988  144.355698         2.86   
3      3.0         700.0    69.550003    778.299988  174.177902         2.22   
4      4.0         700.0    94.330002    900.099976  213.314804        -1.72   
..     ...           ...          ...           ...         ...          ...   
108    8.0   2183.257812    47.490673   1444.360107  650.760376     4.574534   
109    9.0   1896.591187    97.303398   1351.508545  827.040039    -3.222039   
110   10.0   1199.474487   101.517929   1868.645874  393.788116    -0.035194   
111   11.0     903.48175    10.912282    960.143188  602.400879     4.090658   
112   12.0    989.338928    81.168571    798.997742  610.851624     4.238798   

     Intake pressure  Back pressure  In

In [89]:
# Datenaufteilung in Training und Test
from numpy import double


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# X_train = scaler_X.fit_transform(X_train)
# X_test = scaler_X.transform(X_test)

# y_train = scaler_y.fit_transform(y_train)
# y_test = scaler_y.transform(y_test)

x_train = np.array(X_train.values, dtype=float)
x_test = np.array(X_train.values, dtype=float)
y_test = np.array(y_test.values, dtype=float)
y_train = np.array(y_train.values, dtype=float)

# Umwandlung in PyTorch Tensors
X_train = torch.from_numpy(x_train).to(torch.float32)
X_test = torch.from_numpy(x_test).to(torch.float32)
y_train = torch.from_numpy(y_train).to(torch.float32)
y_test = torch.from_numpy(y_test).to(torch.float32)


In [90]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        self.layer2 = nn.Linear(16, 12)
        self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        #print(x[0])
        mat = torch.diag(torch.tensor([1, 1, 1, 1, 1, 1, 1, 1])).to(torch.float32)
        #print(mat[0])
        #x = torch.matmul(x, mat)
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [91]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 3

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [92]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/300], Loss: 2196.4773
Epoch [20/300], Loss: 2608.5247
Epoch [30/300], Loss: 5015.5801
Epoch [40/300], Loss: 215.9424
Epoch [50/300], Loss: 282.3383
Epoch [60/300], Loss: 1278.7155
Epoch [70/300], Loss: 155.9314
Epoch [80/300], Loss: 6436.4492
Epoch [90/300], Loss: 1645.0992
Epoch [100/300], Loss: 257.6303
Epoch [110/300], Loss: 421.2752
Epoch [120/300], Loss: 310894.6562
Epoch [130/300], Loss: 551.7656
Epoch [140/300], Loss: 382.4821
Epoch [150/300], Loss: 2345.7417
Epoch [160/300], Loss: 482.4137
Epoch [170/300], Loss: 5044.0391
Epoch [180/300], Loss: 77.8629
Epoch [190/300], Loss: 366.7238
Epoch [200/300], Loss: 3683.3008
Epoch [210/300], Loss: 67.6539
Epoch [220/300], Loss: 805.7157
Epoch [230/300], Loss: 350.0794
Epoch [240/300], Loss: 3326.8042
Epoch [250/300], Loss: 1079.8528
Epoch [260/300], Loss: 518.3149
Epoch [270/300], Loss: 597.5664
Epoch [280/300], Loss: 238.3482
Epoch [290/300], Loss: 315.8860
Epoch [300/300], Loss: 183.3493


In [93]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    #test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    #print(f'Test Loss: {test_loss.item():.4f}')
    # y_pred = scaler_y.inverse_transform(y_pred)
    # y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

tensor([69.1511, -1.4395, 19.4707, 15.2578, 48.6120]):tensor([165.1238,   3.6310, 104.6658,   7.3947,  99.4201])
tensor([207.5252,   6.8012,  38.9971, 346.1646,  72.4383]):tensor([315.4302,   0.7733,  39.2699,   4.2984,  93.8421])
tensor([77.2682, -0.7009, 24.0888,  6.9952, 47.6075]):tensor([134.1714,   3.2451,  65.6674,  12.9136,  74.5443])
tensor([120.4147,   1.8916,  26.5759,  -7.5148,  45.1551]):tensor([50.6014,  0.5372, 16.3025,  2.3297, 53.8660])
tensor([171.7396,   0.2309,  58.9660,   5.2868,  91.1544]):tensor([22.0959,  0.1985,  3.6819,  1.9589, 45.7711])
tensor([109.0713,  -0.5010,  35.5457,   5.8757,  62.9831]):tensor([94.2330,  2.8329, 38.5090, 10.5363, 59.9567])
tensor([304.9307,   3.5228, 113.0875,  15.6768, 140.2809]):tensor([66.8694,  4.0959, 36.2915, 16.2833, 60.4979])
tensor([146.2139,   0.3528,  50.4721,   5.8361,  76.9466]):tensor([187.1987,   2.8930,  92.9965,   4.8530, 101.0711])
tensor([127.6744,  -3.0959,  35.2981,  13.7912,  90.2829]):tensor([125.6448,   2.2057,

In [94]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_max
    matrix.append(percentage_to_range)
#print(matrix)


In [95]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



14.797601823407158
8.284228243960555
4.063233019071178
1.4860651364149182
11.958741590728104
0.23974098275673733
29.440977922822682
3.4556864421644264
2.4955112239676924
0.7803004071220909
2.5549172038941474
38.048249549081056
Mean:
9.800437795449229


In [96]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

# y_pred_t = np.transpose(y_pred)
# y_test_t = np.transpose(y_test)

# # print(y_pred_t.shape)
# # print(y_test_t.shape)
# featurewise_mean = []
# for feature_t, feature_p in zip(y_test_t, y_pred_t):
#     # print(feature_t)
#     # print(feature_p)
#     # print(mean_squared_error(feature_t, feature_p))
#     featurewise_mean.append(mean_squared_error(feature_t, feature_p))

# print(featurewise_mean)

# #print(y_range)
# featurewise_mean_percentage = featurewise_mean / y_max**2
# print("mse per feature in percentage to value range")
# print(featurewise_mean_percentage)

In [98]:
model.eval()
x = torch.tensor([16.4712565906533, 37.609734587256995, 900.1427871055062, 228.42045796100746, -8.592515890505748, 1981.8799610478413, 3509.5803914574226, 48.709871706251796])
testResults = model(x)
print(type(testResults))
#testResults = scaler_y.inverse_transform(testResults.unsqueeze(0).detach().numpy())
print(testResults)
#[125.10746278858385, 0.4371585551795759, 20.70223941458411, 3.4443235692265173, 102.99879114216813]

<class 'torch.Tensor'>
tensor([182.8054,  -1.4774,  58.2065,  78.3922, 112.2739],
       grad_fn=<ViewBackward0>)
